<a href="https://colab.research.google.com/github/divyar2630/Airbnb/blob/master/modelling_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

## Set filenames

In [22]:
model=pd.read_csv("cust_order_ven__sample20_p2.csv")

In [23]:
## Reduce memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [24]:
model=reduce_mem_usage(model)

Mem. usage decreased to 41.10 Mb (83.0% reduction)


In [25]:
model.columns.values

array(['customer_id', 'cust_location_number', 'Cust_created_days',
       'cust_location_type_Home', 'cust_location_type_Other',
       'cust_location_type_Work', 'gender_female', 'gender_male',
       'cust_lan_long_0', 'cust_lan_long_1', 'cust_lan_long_2',
       'cust_lan_long_3', 'cust_lan_long_4', 'vendor_id',
       'delivery_charge', 'serving_distance', 'is_open',
       'prepration_time', 'discount_percentage', 'ven_status',
       'ven_verified', 'vendor_rating', 'Ven_created_days', 'american',
       'arabic', 'asian', 'bagels', 'biryani', 'breakfast', 'burgers',
       'cafe', 'cakes', 'chinese', 'churros', 'coffee', 'combos',
       'crepes', 'desserts', 'dimsum', 'donuts', 'family_meal',
       'fatayers', 'free_delivery', 'fresh_juices', 'fries',
       'frozen_yoghurt', 'grills', 'healthy_food', 'hot_chocolate',
       'hot_dogs', 'ice_creams', 'indian', 'italian', 'japanese', 'karak',
       'kebabs', 'kids_meal', 'kushari', 'lebanese', 'manakeesh',
       'mandazi', 'm

In [26]:
CV=5 #cv
score='f1' #scoring

In [27]:
model['train_CID X LOC_NUM X VENDOR'].dropna()
model=model.set_index('train_CID X LOC_NUM X VENDOR')


In [28]:
model.isnull().sum().sort_values(ascending=False).head(20)

OpenHour                          27963
Cust_created_days                  7708
Target                                1
timegroup_latenight(23-4.59am)        1
timegroup_evening(17-22.59pm)         1
timegroup_early(5-10.59am)            1
device_type_3                         1
timegroup_normal(11-16.59pm)          1
cafe                                  0
churros                               0
chinese                               0
cakes                                 0
biryani                               0
burgers                               0
breakfast                             0
combos                                0
bagels                                0
coffee                                0
donuts                                0
crepes                                0
dtype: int64

In [33]:
model.drop(['customer_id','OpenHour','Cust_created_days','cust_location_number','vendor_id'],axis=1,inplace=True)

KeyError: ignored

In [13]:
model

,Cust_created_days,cust_location_type_Home,cust_location_type_Other,cust_location_type_Work,gender_female,gender_male,cust_lan_long_0,cust_lan_long_1,cust_lan_long_2,cust_lan_long_3,cust_lan_long_4,delivery_charge,serving_distance,is_open,prepration_time,discount_percentage,ven_status,ven_verified,vendor_rating,Ven_created_days,american,arabic,asian,bagels,biryani,breakfast,burgers,cafe,cakes,chinese,churros,coffee,combos,crepes,desserts,dimsum,donuts,family_meal,fatayers,free_delivery,...,mexican,milkshakes,mishkak,mojitos,mojitos_,nan,omani,organic,pancakes,pasta,pastry,pizza,rice,rolls,salads,sandwiches,seafood,shawarma,shuwa,smoothies,soups,spanish_latte,steaks,sushi,sweets,thai,thali,vegetarian,waffles,OpenHour,TotOpenDuration,vendor_category_en_Restaurants,vendor_category_en_Sweets & Bakes,device_type_1,device_type_3,timegroup_early(5-10.59am),timegroup_evening(17-22.59pm),timegroup_latenight(23-4.59am),timegroup_normal(11-16.59pm),Target
train_CID X LOC_NUM X VENDOR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
LJWO6ZU X 0 X 85,164.0,0,0,0,0,0,1,0,0,0,0,0.000000,5.0,1.0,15,50.0,1.0,1,4.601562,654,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11:59,11.523438,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
EXNH8C6 X 0 X 192,353.0,1,0,0,0,0,0,1,0,0,0,0.700195,10.0,1.0,11,0.0,1.0,1,4.300781,482,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,09:00,14.750000,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
FMUYXT3 X 0 X 676,197.0,1,0,0,0,0,0,0,1,0,0,0.700195,14.0,1.0,10,0.0,1.0,1,4.398438,227,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,09:00,13.000000,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
PXAQC4C X 0 X 298,153.0,0,0,0,0,0,1,0,0,0,0,0.000000,8.0,1.0,17,0.0,1.0,1,4.699219,369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10:00,13.000000,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
LV8K1WS X 0 X 299,290.0,0,0,1,0,1,0,1,0,0,0,0.700195,15.0,1.0,15,0.0,1.0,1,4.199219,368,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10:00,3.750000,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NWEE5JF X 0 X 231,227.0,1,0,0,0,1,1,0,0,0,0,0.700195,10.0,1.0,15,0.0,1.0,1,4.300781,434,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,09:00,14.000000,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
JX70DD7 X 0 X 855,250.0,1,0,0,0,1,1,0,0,0,0,0.000000,8.0,1.0,10,0.0,1.0,1,4.199219,193,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4Y0K8NQ X 3 X 43,243.0,1,0,0,0,0,1,0,0,0,0,0.700195,15.0,1.0,15,0.0,1.0,1,4.300781,742,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11:00,12.750000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Separate predictors and response
X = model.drop('Target', axis = 1)
y = model['Target']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [12]:
Gradient_boosting=GradientBoostingClassifier(min_samples_leaf = 5, random_state = 1, learning_rate = 0.01)
paras_5= {'n_estimators': np.linspace(10, 100, dtype = int)}
BoT = GridSearchCV(estimator=Gradient_boosting,param_grid = paras_5, cv = CV, n_jobs = -1, scoring = score)
BoT.fit(X_train, y_train)

ValueError: ignored

In [ ]:
BoT_f1=f1_score(y_holdout,BoT.predict(X_holdout))
BoT_f1

In [ ]:
#convert to csv file
cust_order_ven.to_csv("cust_order_ven_p2.csv",index=False)